In [2]:
import numpy as np
import cvxpy as cp
import pickle

from problems.data_generation_Price_Setting import generate_problem_data
from problems.problem_definition_Price_Setting import BilevelProblem
from problems.problem_definition_Price_Setting_regularized import BilevelProblem_regularized
from algorithms.barrier_blo import BarrierBLO
from algorithms.blocc import BLOCC
from algorithms.implicit_gradient_descent import IGD
from algorithms.BiC_GAFFA import BiC_GAFFA
from algorithms.BSG import BSG

n = 60
hparams = {
    'barrier_blo': {
        'M': 0.00001,
        't': 0.0001,
        'alpha_xy': 0.01,
        'alpha_T': 0.02,
        'epsilon_xy': 0.01,
        'epsilon_T': 0.0005,
        'inner_max_iters': 1000000,
        'outer_max_iters': 100000
    },
    'blocc': {
        'gamma': 2,
        'alpha_T': 0.01,
        'alpha_g_xy': 30,
        'alpha_F_xy': 30,
        'beta_g_xy': 0.00002,
        'beta_F_xy': 0.00002,
        'epsilon_T': 0.00005,
        'epsilon_inner_xy_g': 0.0001,
        'epsilon_outer_xy_g': 1,
        'epsilon_inner_xy_F': 0.0001,
        'epsilon_outer_xy_F': 1,
        'maxmin_g_outer_max_iters': 5000000,
        'maxmin_F_outer_max_iters': 5000000,
        'maxmin_g_inner_max_iters': 5000000,
        'maxmin_F_inner_max_iters': 5000000,
        'main_max_iters': 10000000000000
    },
    'BSG':{
        'alpha_T':0.01,
        'alpha_xy':30,
        'alpha_z':0.00002,
        'epsilon_T':0.0005,
        'epsilon_xy':0.0005,
        'epsilon_z':1,
        'max_iters_T':1000000,
        'max_iters_xy':1000000,
        'max_iters_z':1000000
    }
}

seed = 9

data = generate_problem_data(n, seed)
problem = BilevelProblem(data)
problem_regularized = BilevelProblem_regularized(data)

max_elapsed_time = 240
T_init = np.zeros(n)
x_init = np.zeros(n)
y_init = np.zeros(n)
z_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
theta_init = np.zeros(n)
x_g_init = np.zeros(n)
x_F_init = np.zeros(n)
y_g_init = np.zeros(n)
y_F_init = np.zeros(n)
mu_g_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
mu_F_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)

In [ ]:
n = 60
max_elapsed_time = 240
for seed in range(2, 3):
    data = generate_problem_data(n, seed)
    problem = BilevelProblem(data)
    problem_regularized = BilevelProblem_regularized(data)

    T_init = np.zeros(n)
    x_init = np.zeros(n)
    y_init = np.zeros(n)
    z_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
    theta_init = np.zeros(n)
    x_g_init = np.zeros(n)
    x_F_init = np.zeros(n)
    y_g_init = np.zeros(n)
    y_F_init = np.zeros(n)
    mu_g_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
    mu_F_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
    
    ## barrier algo
    barrier_algo = BarrierBLO(problem, hparams)
    T_opt_barrier, x_opt_barrier, y_opt_barrier, history_barrier = barrier_algo.upper_loop(T_init, x_init, y_init, max_elapsed_time, step_size_type="diminish")
    f_barrier_num = problem.f(T_opt_barrier, x_opt_barrier, y_opt_barrier)
    print(f_barrier_num)
    
    x_original_opt_barrier, y_original_opt_barrier = barrier_algo.Interior_inner_loop(T_opt_barrier)
    f_barrier_theo = problem.f(T_opt_barrier, x_original_opt_barrier, y_original_opt_barrier)
    g_barrier_theo = problem.g(T_opt_barrier, x_original_opt_barrier, y_original_opt_barrier)
    g_barrier_num = problem.g(T_opt_barrier, x_opt_barrier, y_opt_barrier)

    h1_values = [problem.h_1(T_opt_barrier, x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h1)]
    h2_values = [problem.h_2(T_opt_barrier, x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h2)]
    h_total_barrier = sum(max(0, val) for val in h1_values + h2_values)

    # print(f"x_opt_barrier={x_original_opt_barrier},y_opt_barrier={y_original_opt_barrier}")

    print(f"f_theoretical_opt_result={f_barrier_theo}, f_barrier_numerical_result={f_barrier_num}")
    print(f"g_theoretical_opt_result={g_barrier_theo}, g_barrier_numerical_result={g_barrier_num}")
    print(f"feasibility_violation={h_total_barrier}")
    
    # ## BLOCC
    # blocc_algo = BLOCC(problem_regularized, hparams)
    # x_opt_blocc, y_opt_blocc, history_blocc = blocc_algo.blocc(T_init, x_g_init, x_F_init, y_g_init, y_F_init, mu_g_init, mu_F_init, max_elapsed_time)
    # f_blocc_num = problem.f(x_opt_blocc, y_opt_blocc)
    # print(f_blocc_num)
    
    ## saving results
    results = {
        'barrier': {
            'x_opt': x_opt_barrier,
            'y_opt': y_opt_barrier,
            'history': history_barrier,
            'y_original_opt': y_original_opt_barrier,
            'f_theoretical_opt': f_barrier_theo,
            'f_numerical': f_barrier_num,
            'g_theoretical_opt': g_barrier_theo,
            'g_numerical': g_barrier_num,
            'h_total': h_total_barrier
        }
    }

    save_filename = f'results_PS_seed_barrier_{seed}.pkl'

    with open(save_filename, 'wb') as f:
        pickle.dump(results, f)

    print(f"Results saved to {save_filename}")

In [ ]:
n = 60
max_elapsed_time = 240
for seed in range(10):
    data = generate_problem_data(n, seed)
    problem = BilevelProblem(data)
    problem_regularized = BilevelProblem_regularized(data)

    T_init = np.zeros(n)
    x_init = np.zeros(n)
    y_init = np.zeros(n)
    z_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
    theta_init = np.zeros(n)
    x_g_init = np.zeros(n)
    x_F_init = np.zeros(n)
    y_g_init = np.zeros(n)
    y_F_init = np.zeros(n)
    mu_g_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
    mu_F_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
    
    ## barrier algo
    # barrier_algo = BarrierBLO(problem, hparams)
    # T_opt_barrier, x_opt_barrier, y_opt_barrier, history_barrier = barrier_algo.upper_loop(T_init, x_init, y_init, max_elapsed_time)
    # f_barrier_num = problem.f(T_opt_barrier, x_opt_barrier, y_opt_barrier)
    # print(f_barrier_num)
    
    # x_original_opt_barrier, y_original_opt_barrier = barrier_algo.Interior_inner_loop(T_opt_barrier)
    # f_barrier_theo = problem.f(T_opt_barrier, x_original_opt_barrier, y_original_opt_barrier)
    # g_barrier_theo = problem.g(T_opt_barrier, x_original_opt_barrier, y_original_opt_barrier)
    # g_barrier_num = problem.g(T_opt_barrier, x_opt_barrier, y_opt_barrier)

    # h1_values = [problem.h_1(T_opt_barrier, x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h1)]
    # h2_values = [problem.h_2(T_opt_barrier, x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h2)]
    # h_total_barrier = sum(max(0, val) for val in h1_values + h2_values)

    # # print(f"x_opt_barrier={x_original_opt_barrier},y_opt_barrier={y_original_opt_barrier}")

    # print(f"f_theoretical_opt_result={f_barrier_theo}, f_barrier_numerical_result={f_barrier_num}")
    # print(f"g_theoretical_opt_result={g_barrier_theo}, g_barrier_numerical_result={g_barrier_num}")
    # print(f"feasibility_violation={h_total_barrier}")
    
    # ## BLOCC
    # blocc_algo = BLOCC(problem_regularized, hparams)
    # x_opt_blocc, y_opt_blocc, history_blocc = blocc_algo.blocc(T_init, x_g_init, x_F_init, y_g_init, y_F_init, mu_g_init, mu_F_init, max_elapsed_time)
    # f_blocc_num = problem.f(x_opt_blocc, y_opt_blocc)
    # print(f_blocc_num)
    
    ## BSG
    barrier_algo = BarrierBLO(problem, hparams)
    bsg_algo = BSG(problem_regularized, hparams)
    T_opt_bsg, x_opt_bsg, y_opt_bsg, history_bsg = bsg_algo.bsg(T_init, x_init, y_init, z_init, max_elapsed_time, step_size_type="diminish")
    f_bsg_num = problem.f(T_opt_bsg, x_opt_bsg, y_opt_bsg)
    print(f_bsg_num)

    x_original_opt_bsg, y_original_opt_bsg = barrier_algo.Interior_inner_loop(T_opt_bsg)
    f_bsg_theo = problem.f(T_opt_bsg, x_original_opt_bsg, y_original_opt_bsg)
    g_bsg_theo = problem.g(T_opt_bsg, x_original_opt_bsg, y_original_opt_bsg)
    g_bsg_num = problem.g(T_opt_bsg, x_opt_bsg, y_opt_bsg)

    h1_values = [problem.h_1(T_opt_bsg, x_opt_bsg, y_opt_bsg, i) for i in range(problem.num_constraints_h1)]
    h2_values = [problem.h_2(T_opt_bsg, x_opt_bsg, y_opt_bsg, i) for i in range(problem.num_constraints_h2)]
    h_total_bsg = sum(max(0, val) for val in h1_values + h2_values)

    print(f"f_theoretical_opt_result={f_bsg_theo}, f_bsg_numerical_result={f_bsg_num}")
    print(f"g_theoretical_opt_result={g_bsg_theo}, g_bsg_numerical_result={g_bsg_num}")
    print(h_total_bsg)
    
    ## saving results
    results = {
        'bsg': {
            'x_opt': x_opt_bsg,
            'y_opt': y_opt_bsg,
            'history': history_bsg,
            'y_original_opt': y_original_opt_bsg,
            'f_theoretical_opt': f_bsg_theo,
            'f_numerical': f_bsg_num,
            'g_theoretical_opt': g_bsg_theo,
            'g_numerical': g_bsg_num,
            'h_total': h_total_bsg
        }
    }

    save_filename = f'results_PS_seed_bsg_{seed}.pkl'

    with open(save_filename, 'wb') as f:
        pickle.dump(results, f)

    print(f"Results saved to {save_filename}")

In [ ]:
n = 60
max_elapsed_time = 240
for seed in range(10):
    data = generate_problem_data(n, seed)
    problem = BilevelProblem(data)
    problem_regularized = BilevelProblem_regularized(data)

    T_init = np.zeros(n)
    x_init = np.zeros(n)
    y_init = np.zeros(n)
    z_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
    theta_init = np.zeros(n)
    x_g_init = np.zeros(n)
    x_F_init = np.zeros(n)
    y_g_init = np.zeros(n)
    y_F_init = np.zeros(n)
    mu_g_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
    mu_F_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
    
    ## barrier algo
    # barrier_algo = BarrierBLO(problem, hparams)
    # T_opt_barrier, x_opt_barrier, y_opt_barrier, history_barrier = barrier_algo.upper_loop(T_init, x_init, y_init, max_elapsed_time)
    # f_barrier_num = problem.f(T_opt_barrier, x_opt_barrier, y_opt_barrier)
    # print(f_barrier_num)
    
    # x_original_opt_barrier, y_original_opt_barrier = barrier_algo.Interior_inner_loop(T_opt_barrier)
    # f_barrier_theo = problem.f(T_opt_barrier, x_original_opt_barrier, y_original_opt_barrier)
    # g_barrier_theo = problem.g(T_opt_barrier, x_original_opt_barrier, y_original_opt_barrier)
    # g_barrier_num = problem.g(T_opt_barrier, x_opt_barrier, y_opt_barrier)

    # h1_values = [problem.h_1(T_opt_barrier, x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h1)]
    # h2_values = [problem.h_2(T_opt_barrier, x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h2)]
    # h_total_barrier = sum(max(0, val) for val in h1_values + h2_values)

    # # print(f"x_opt_barrier={x_original_opt_barrier},y_opt_barrier={y_original_opt_barrier}")

    # print(f"f_theoretical_opt_result={f_barrier_theo}, f_barrier_numerical_result={f_barrier_num}")
    # print(f"g_theoretical_opt_result={g_barrier_theo}, g_barrier_numerical_result={g_barrier_num}")
    # print(f"feasibility_violation={h_total_barrier}")
    
    ## BLOCC
    barrier_algo = BarrierBLO(problem, hparams)
    blocc_algo = BLOCC(problem_regularized, hparams)
    T_opt_blocc, x_opt_blocc, y_opt_blocc, history_blocc = blocc_algo.blocc(T_init, x_g_init, x_F_init, y_g_init, y_F_init, mu_g_init, mu_F_init, max_elapsed_time, step_size_type="diminish")
    f_blocc_num = problem.f(T_opt_blocc, x_opt_blocc, y_opt_blocc)
    print(f_blocc_num)
    
    # ## BSG
    # barrier_algo = BarrierBLO(problem, hparams)
    # bsg_algo = BSG(problem_regularized, hparams)
    # T_opt_bsg, x_opt_bsg, y_opt_bsg, history_bsg = bsg_algo.bsg(T_init, x_init, y_init, z_init, max_elapsed_time, step_size_type="diminish")
    # f_bsg_num = problem.f(T_opt_bsg, x_opt_bsg, y_opt_bsg)
    # print(f_bsg_num)

    x_original_opt_blocc, y_original_opt_blocc = barrier_algo.Interior_inner_loop(T_opt_blocc)
    f_blocc_theo = problem.f(T_opt_blocc, x_original_opt_blocc, y_original_opt_blocc)
    g_blocc_theo = problem.g(T_opt_blocc, x_original_opt_blocc, y_original_opt_blocc)
    g_blocc_num = problem.g(T_opt_blocc, x_opt_blocc, y_opt_blocc)

    h1_values = [problem.h_1(T_opt_blocc, x_opt_blocc, y_opt_blocc, i) for i in range(problem.num_constraints_h1)]
    h2_values = [problem.h_2(T_opt_blocc, x_opt_blocc, y_opt_blocc, i) for i in range(problem.num_constraints_h2)]
    h_total_blocc = sum(max(0, val) for val in h1_values + h2_values)

    print(f"f_theoretical_opt_result={f_blocc_theo}, f_blocc_numerical_result={f_blocc_num}")
    print(f"g_theoretical_opt_result={g_blocc_theo}, g_blocc_numerical_result={g_blocc_num}")
    print(h_total_blocc)
    
    ## saving results
    results = {
        'blocc': {
            'x_opt': x_opt_blocc,
            'y_opt': y_opt_blocc,
            'history': history_blocc,
            'y_original_opt': y_original_opt_blocc,
            'f_theoretical_opt': f_blocc_theo,
            'f_numerical': f_blocc_num,
            'g_theoretical_opt': g_blocc_theo,
            'g_numerical': g_blocc_num,
            'h_total': h_total_blocc
        }
    }

    save_filename = f'results_PS_seed_blocc_{seed}.pkl'

    with open(save_filename, 'wb') as f:
        pickle.dump(results, f)

    print(f"Results saved to {save_filename}")

In [ ]:
for seed in range(10):
    filename = f'results_PS_seed_barrier_{seed}.pkl'
    with open(filename, 'rb') as f:
        results = pickle.load(f)
    history = results['barrier']['history']
    if history:
        last_time = history[-1]['time']
        f_num = results['barrier']['f_numerical']
        f_theo = results['barrier']['f_theoretical_opt']
        g_numerical = results['barrier']['g_numerical']
        g_theoretical_opt = results['barrier']['g_theoretical_opt']
        h_total = results['barrier']['h_total']
        print(f"Seed {seed} - Total Time: {last_time:.2f} second, Upper-level function value: {f_num}, Hyperfunction: {f_theo}, Optimality Gap: {g_numerical - g_theoretical_opt:.4f}, Total Violation: {h_total}")

In [ ]:
for seed in range(10):
    filename = f'results_PS_seed_bsg_{seed}.pkl'
    with open(filename, 'rb') as f:
        results = pickle.load(f)
    history = results['bsg']['history']
    if history:
        last_time = history[-1]['time']
        f_num = results['bsg']['f_numerical']
        f_theo = results['bsg']['f_theoretical_opt']
        g_numerical = results['bsg']['g_numerical']
        g_theoretical_opt = results['bsg']['g_theoretical_opt']
        h_total = results['bsg']['h_total']
        print(f"Seed {seed} - Total Time: {last_time:.2f} second, Upper-level function value: {f_num}, Hyperfunction: {f_theo}, Optimality Gap: {g_numerical - g_theoretical_opt:.4f}, Total Violation: {h_total}")

In [ ]:
for seed in range(10):
    filename = f'results_PS_seed_blocc_{seed}.pkl'
    with open(filename, 'rb') as f:
        results = pickle.load(f)
    history = results['blocc']['history']
    if history:
        last_time = history[-1]['time']
        f_num = results['blocc']['f_numerical']
        f_theo = results['blocc']['f_theoretical_opt']
        g_numerical = results['blocc']['g_numerical']
        g_theoretical_opt = results['blocc']['g_theoretical_opt']
        h_total = results['blocc']['h_total']
        print(f"Seed {seed} - Total Time: {last_time:.2f} second, Upper-level function value: {f_num}, Hyperfunction: {f_theo}, Optimality Gap: {g_numerical - g_theoretical_opt:.4f}, Total Violation: {h_total}")

In [ ]:
hparams['barrier_blo'] = {
            'M': 0.00001,
            't': 0.0001,
            'alpha_xy': 0.01,
            'alpha_T': 0.02,
            'epsilon_xy': 0.01,
            'epsilon_T': 0.0005,
            'inner_max_iters': 1000000,
            'outer_max_iters': 100000
        }

max_elapsed_time = 480
barrier_algo = BarrierBLO(problem, hparams)
T_opt_barrier, x_opt_barrier, y_opt_barrier, history_barrier = barrier_algo.upper_loop(T_init, x_init, y_init, max_elapsed_time, step_size_type="diminish")
f_barrier_num = problem.f(T_opt_barrier, x_opt_barrier, y_opt_barrier)
print(f_barrier_num)


In [ ]:
f_barrier_num = problem.f(T_opt_barrier, x_opt_barrier, y_opt_barrier)
print(f_barrier_num)
    
barrier_algo = BarrierBLO(problem, hparams)
x_original_opt_barrier, y_original_opt_barrier = barrier_algo.Interior_inner_loop(T_opt_barrier)
print(T_opt_barrier)
print(x_original_opt_barrier)
f_barrier_theo = problem.f(T_opt_barrier, x_original_opt_barrier, y_original_opt_barrier)
g_barrier_theo = problem.g(T_opt_barrier, x_original_opt_barrier, y_original_opt_barrier)
g_barrier_num = problem.g(T_opt_barrier, x_opt_barrier, y_opt_barrier)

h1_values = [problem.h_1(T_opt_barrier, x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(T_opt_barrier, x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h2)]
h_total_barrier = sum(max(0, val) for val in h1_values + h2_values)

print(f"f_theoretical_opt_result={f_barrier_theo}, f_bsg_numerical_result={f_barrier_num}")
print(f"g_theoretical_opt_result={g_barrier_theo}, g_bsg_numerical_result={g_barrier_num}")
print(h_total_barrier)

In [ ]:
hparams['blocc'] = {
            'gamma': 2,
            'alpha_T': 0.001,
            'alpha_g_xy': 30,
            'alpha_F_xy': 30,
            'beta_g_xy': 0.00002,
            'beta_F_xy': 0.00002,
            'epsilon_T': 0.00005,
            'epsilon_inner_xy_g': 0.0001,
            'epsilon_outer_xy_g': 1,
            'epsilon_inner_xy_F': 0.0001,
            'epsilon_outer_xy_F': 1,
            'maxmin_g_outer_max_iters': 5000000,
            'maxmin_F_outer_max_iters': 5000000,
            'maxmin_g_inner_max_iters': 5000000,
            'maxmin_F_inner_max_iters': 5000000,
            'main_max_iters': 10000000000000
        }

max_elapsed_time = 240
blocc_algo = BLOCC(problem_regularized, hparams)
T_opt_blocc, x_opt_blocc, y_opt_blocc, history_blocc = blocc_algo.blocc(T_init, x_g_init, x_F_init, y_g_init, y_F_init, mu_g_init, mu_F_init, max_elapsed_time, step_size_type="diminish")
f_blocc_num = problem.f(T_opt_blocc, x_opt_blocc, y_opt_blocc)
print(f_blocc_num)

In [ ]:
hparams['BSG'] = {
            'alpha_T':0.01,
            'alpha_xy':30,
            'alpha_z':0.00002,
            'epsilon_T':0.0005,
            'epsilon_xy':0.0001,
            'epsilon_z':0.5,
            'max_iters_T':1000000,
            'max_iters_xy':1000000,
            'max_iters_z':1000000
        }

max_elapsed_time = 480
bsg_algo = BSG(problem_regularized, hparams)
T_opt_bsg, x_opt_bsg, y_opt_bsg, history_bsg = bsg_algo.bsg(T_init, x_init, y_init, z_init, max_elapsed_time, step_size_type="diminish")
f_bsg_num = problem.f(T_opt_bsg, x_opt_bsg, y_opt_bsg)
print(f_bsg_num)

In [ ]:
f_bsg_num = problem.f(T_opt_bsg, x_opt_bsg, y_opt_bsg)
print(f_bsg_num)
    
barrier_algo = BarrierBLO(problem, hparams)
x_original_opt_bsg, y_original_opt_bsg = barrier_algo.Interior_inner_loop(T_opt_bsg)
print(T_opt_bsg)
print(x_original_opt_bsg)
f_bsg_theo = problem.f(T_opt_bsg, x_original_opt_bsg, y_original_opt_bsg)
g_bsg_theo = problem.g(T_opt_bsg, x_original_opt_bsg, y_original_opt_bsg)
g_bsg_num = problem.g(T_opt_bsg, x_opt_bsg, y_opt_bsg)

h1_values = [problem.h_1(T_opt_bsg, x_opt_bsg, y_opt_bsg, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(T_opt_bsg, x_opt_bsg, y_opt_bsg, i) for i in range(problem.num_constraints_h2)]
h_total_bsg = sum(max(0, val) for val in h1_values + h2_values)

print(f"f_theoretical_opt_result={f_bsg_theo}, f_bsg_numerical_result={f_bsg_num}")
print(f"g_theoretical_opt_result={g_bsg_theo}, g_bsg_numerical_result={g_bsg_num}")
print(h_total_bsg)

In [ ]:
f_blocc_num = problem.f(T_opt_blocc, x_opt_blocc, y_opt_blocc)
print(f_blocc_num)
    
barrier_algo = BarrierBLO(problem, hparams)
x_original_opt_blocc, y_original_opt_blocc = barrier_algo.Interior_inner_loop(T_opt_blocc)
print(T_opt_blocc)
print(x_original_opt_blocc)
f_blocc_theo = problem.f(T_opt_blocc, x_original_opt_blocc, y_original_opt_blocc)
g_blocc_theo = problem.g(T_opt_blocc, x_original_opt_blocc, y_original_opt_blocc)
g_blocc_num = problem.g(T_opt_blocc, x_opt_blocc, y_opt_blocc)

h1_values = [problem.h_1(T_opt_blocc, x_opt_blocc, y_opt_blocc, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(T_opt_blocc, x_opt_blocc, y_opt_blocc, i) for i in range(problem.num_constraints_h2)]
h_total_blocc = sum(max(0, val) for val in h1_values + h2_values)

print(f"f_theoretical_opt_result={f_blocc_theo}, f_blocc_numerical_result={f_blocc_num}")
print(f"g_theoretical_opt_result={g_blocc_theo}, g_blocc_numerical_result={g_blocc_num}")
print(h_total_blocc)